In [ ]:
#이 프로그램은 Ensemble 테스트 전용으로 annotation 처리는 제외한다.
#신규방식인 각 모델 결과를 모아서 최적의 영역을 선택 추출
#이전 방식(한 모델의 결과를 걸러 다음 모델로 넘겨 테스트)은 new_RetinaNet - TEST.ipynb 참조

%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
from itertools import groupby
from itertools import chain
from operator import eq
import keras
import keras.preprocessing.image
from keras_retinanet.models.resnet import custom_objects
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
from keras_retinanet.preprocessing.dicom2jpg_generator import DICOM2JPG_Generator
import matplotlib.pyplot as plt
import cv2
import numpy as np
import time
import tensorflow as tf
import keras_retinanet.utils.new_util as my

OS_ENV = 'LINUX' # 'LINUX' or 'WIN'
MODEL_ENV = 'ensemble' # 'ensemble' or 'separate'
DATE = '20181122_from_dicom_with_merge'
DATA_TO_TEST = 'abn_all' # 'abn1' or 'abn2' or 'abn_all' or 'normal' or 'normal_1'
#normal_1': #2018. 1월말 데이터중 Normal 1318 file
DATA_SOURCE_TYPE = '_MEM_' #메모리 = _MEM_, 파일 = _FILE_

#SCORE_VALUE_LIST = [0.5,0.4]
SCORE_VALUE_LIST = [0.4]

ordinal=lambda x:["1st","2nd","3rd","4th","5th","6th"][x-1]

In [ ]:
if OS_ENV == 'LINUX':
    app_base_path = '/home/huray/workspace/han_work/han_test'
    data_base_path = "/home/huray/data/NCC"
else:
    app_base_path = 'D:/Work/NCC/han_test'
    data_base_path = "D:/Work/NCC/data/NCC"

#----test file list for detect-----#
test_file_list = [] #전체 테스트 파일 목록, full path
test_file_name_list = [] #전체 데스트 파일명 목록, path를 제외, 폴더명+파일명 조함으로 unique한 name 구성
num_of_test_files = 0 #전체 테스대상 파일 개수

#--------------- dicom 관련 ----------------------------#
#dicom 및 jpg file path    
dicom_base_path = os.path.join(data_base_path, "dicom_test")
#jpg_base_path = dicom_base_path.replace('dicom', 'img_retinanet')
jpg_base_path = os.path.join(app_base_path, 'jpg_from_dicom')
dicom_file_path = os.path.join(dicom_base_path, "abn*/**/*.dcm")
if 'normal'in DATA_TO_TEST:
    dicom_file_path = os.path.join(dicom_base_path, "normal*/**/*.dcm")    
    
jpg_sub_path = 'jpg'
#dicom에서 읽은 정보를 jpg로 저장할 경우
img_save_path = os.path.join(jpg_base_path, jpg_sub_path)

if '_FILE_' in DATA_SOURCE_TYPE and not os.path.exists(img_save_path):
    os.makedirs(img_save_path)
#-------------------------------------------------------#

#----------- test 결과 save path -----------------------#
result_save_base_path = os.path.join(app_base_path, 'results')
result_save_subpath = os.path.join(DATE + '_' + MODEL_ENV, DATA_TO_TEST)
result_save_path = os.path.join(result_save_base_path, result_save_subpath)
if not os.path.exists(result_save_path):
    os.makedirs(result_save_path)
#------------------------------------------------------#    

#-------------------- test결과 정보관련 --------------------#
number_abnormal_detect_from_normal = 0 #normal 파일에서 abnormal로 검출된 경우의 수
####for 테스트 파일목록 Groupping
groupby_key_list = [] #그룹명(테스트 파일 단위 그룹 명) 목록
groupby_file_list = [] #그룹화된 테스트파일 목록
groupby_detected_number_for_files = [] #검출(detect)된 수에 대한 그룹화된 목록(그룹화된 테스트 목록 구조와 동일하게 매핑)
#list_not_detected_group_index = [] #검출되지 않은 그룹의 index 목록 
#M_max_scores = [] #abnormal 테스트 파일에서 detect된 anchor score 중 가장 높은 score
#N_max_scores = [] #normal 각 테스트 파일에서 detect된 anchor score 중 가장 높은 score
#----------------------------------------------------------#

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
keras.backend.tensorflow_backend.set_session(get_session())

## Load RetinaNet model

In [ ]:
model_list = []
#resnet50
#model_list.append('snapshots/1001_resnet50_ep100_with_valid/resnet50_csv_84.h5')
#model_list.append('snapshots/1001_resnet50_ep100_with_valid/1017_snapshot_resnet50_csv_90_ep100_with_valid/resnet50_csv_03.h5')
#model_list.append('snapshots/1001_resnet50_ep100_with_valid/1017_snapshot_resnet50_csv_90_ep100_with_valid/resnet50_csv_19.h5')
#model_list.append('snapshots/1001_resnet50_ep100_with_valid/1017_snapshot_resnet50_csv_90_ep100_with_valid/resnet50_csv_27.h5')
model_list.append('snapshots/1001_resnet50_ep100_with_valid/1017_snapshot_resnet50_csv_90_ep100_with_valid/resnet50_csv_36.h5')

#resnet101
#model_list.append('snapshots/0928_resnet101_ep100_with_valid/resnet101_csv_61.h5')
###model_list.append('snapshots/0928_resnet101_ep100_with_valid/resnet101_csv_59.h5')
model_list.append('snapshots/0928_resnet101_ep100_with_valid//1030_snapshot_resnet101_csv_61_ep100_with_valid/resnet101_csv_35.h5')
#model_list.append('snapshots/0928_resnet101_ep100_with_valid//1030_snapshot_resnet101_csv_61_ep100_with_valid/resnet101_csv_29.h5')
#model_list.append('snapshots/0928_resnet101_ep100_with_valid//1030_snapshot_resnet101_csv_61_ep100_with_valid/resnet101_csv_56.h5')

#resnet152
#model_list.append('snapshots/0927_resnet152_ep100_with_valid/resnet152_csv_74.h5')
#model_list.append('snapshots/0927_resnet152_ep100_with_valid/1012_snapshot_resnet152_csv_74_ep100_with_valid/resnet152_csv_32.h5')
model_list.append('snapshots/0927_resnet152_ep100_with_valid/1012_snapshot_resnet152_csv_74_ep100_with_valid/resnet152_csv_45.h5')
#model_list.append('snapshots/0927_resnet152_ep100_with_valid/1012_snapshot_resnet152_csv_74_ep100_with_valid/resnet152_csv_53.h5')
#model_list.append('snapshots/0927_resnet152_ep100_with_valid/1012_snapshot_resnet152_csv_74_ep100_with_valid/resnet152_csv_57.h5')
#model_list.append('snapshots/0927_resnet152_ep100_with_valid/1012_snapshot_resnet152_csv_74_ep100_with_valid/resnet152_csv_84.h5')


####################################### 이전모델 ###################################
#ensemble 아래 e44 + e29
#single e29
#model_list.append('snapshots/0326_more_aug/resnet101_csv_44.h5')   
#model_list.append('snapshots/0525_add_ncc_mass_data_strong_aug/resnet101_csv_29.h5')
#######

#model_list.append('snapshots/0620_stronger_aug/resnet101_csv_25.h5')
#model_list.append('snapshots/0620_stronger_aug/resnet101_csv_29.h5')
#model_list.append('snapshots/0620_stronger_aug/resnet101_csv_47.h5')
#model_list.append('snapshots/0620_stronger_aug/resnet101_csv_48.h5')
#model_list.append('snapshots/0620_stronger_aug/resnet101_csv_49.h5')

## Initialize data generators

In [ ]:
if OS_ENV == 'LINUX':
    label_csv_path = '/home/huray/data/new_trainset/class_2_malig_or_norm.csv'
else:
    label_csv_path = 'D:/Work/NCC/data/new_trainset/class_2_malig_or_norm.csv'    
    
# create a dicom2jpg generator for testing data
jpg_generator = DICOM2JPG_Generator(
    label_csv_path,
    dicom_file_path,
    jpg_base_path,
    img_save_path,
    with_anno = False
)

## define and initialize

In [ ]:
#dicom 원본  path에서 유일한 파일명을 구성한다.
def make_file_name(m_path):
    _split = m_path.split('/')
    _name = '{}-{}-{}-{}'.format(_split[-4], _split[-3], _split[-2], _split[-1].replace('.dcm', ''))
    return _name
    
############################# !!!!!!!!!!!!!!!!!!!!!!!!!!! ############################# 
###테스트 결과 출력
def get_detection_info():
    list_not_detected_group_index = []  # 검출되지 않은 그룹의 index 목록
    list_not_detected_group_index_by_annotation = [] # annotation 일치 조건으로 검출되지 않은 그룹의 index 목록
    total_not_detected_file = 0 #검출되지 않은 전체 테스트 파일의 수    
    i_group_member = 0

    print('------------------------------------------------ Result info --------------------------------------------------------')
    #print('\n')    
    
    for idx_x, row in enumerate(groupby_detected_number_for_files):
        i_detections = 0
        for idx_y, elem in enumerate(row):
            i_detections += elem
            if elem == 0:
                total_not_detected_file += 1
                
        if i_detections == 0:
            list_not_detected_group_index.append(idx_x)
            i_group_member += len(row)
            print('({})set have no detection!.'.format(groupby_key_list[idx_x]))

    #print('\n')

    #1st print type
    #print('Total Number of abnormal case not detected :::::::::: {}'.format(total_not_detected_file))
    #print('Number of abnormal case set not detected::::::: {}({} files)'.format(len(list_not_detected_group_index), i_group_member))

    #2nd print type
    #print('{}/{} files not detected.'.format(total_not_detected_file,len(test_file_list)))
    #print('{}({} files)/{} sets not detected.'.format(len(list_not_detected_group_index), i_group_member, len(groupby_key_list)))

    #3rd print type
    print('{} files'.format(total_not_detected_file))
    print('{}({} files) sets  not detected.'.format(len(list_not_detected_group_index), i_group_member))
    #print('\n')
    if 'normal' in DATA_TO_TEST:
        print('{} abnormal detected.(from {} normal files).'.format(number_abnormal_detect_from_normal,len(test_file_list)))
    
    print('---------------------------------------------------------------------------------------------------------------------')

###정보 초기화(일부 list 정보에 대한 초기화)
def init_info():
    global groupby_detected_number_for_files
    global number_abnormal_detect_from_normal
    for idx_i, row in enumerate(groupby_file_list):
        for idx_j, col in enumerate(row):
            groupby_detected_number_for_files[idx_i][idx_j] = 0
            
    #set_anno_file_index()
    number_abnormal_detect_from_normal = 0
    
            
###그룹화된 테스트 파일 구조에서 annotation 정보가 있는 파일의 위치를 annotation dict 정보에 set
def set_anno_file_index():
    global annotation_detect_num_dict
    for key in annotation_detect_num_dict:
        annotation_detect_num_dict[key] = 0
              
                
#---------------------------------------------------------------------------------------------    
#path포함한 파일 목록과, 파일명만(확장자도 제외, 유일성을 위해서 일부 directory명을 포함하여 구성)
test_file_list = jpg_generator.dicom_files() #검출 대상 dicom file 목록
num_of_test_files = len(test_file_list)    
print('total test file : {}'.format(num_of_test_files))

#검출대상 파일 목록에 대해 file name, annotation 정보등을 얻는다.
for mammo_path in test_file_list:
    test_file_name_list.append(make_file_name(mammo_path))
'''    
    _annos = jpg_generator.read_annotations(mammo_path)
    if len(_annos) > 0:
        anno_file_list.append(make_file_name(mammo_path))
        annotation_num_dict[make_file_name(mammo_path)] = len(_annos)
        annotation_detect_num_dict[make_file_name(mammo_path)] = 0
'''
#print('annotation_pos_dict : {}'.format(annotation_pos_dict))
#print('annotation_num_dict : {}'.format(annotation_num_dict))
        
    
########## 테스트 파일목록 정보 set  ######
#테스트 파일 목록을 groupby 하기 위해서 정렬
test_file_name_list.sort(key=lambda x: '-'.join(x.split('-')[:3]))

#테스트 파일 목록을 group by하여 list에 저장
#groupby key(그룹핑 기준(단위)는 파일명을 '-'로 split 한뒤 앞쪽 3번째 요소까지 추출함)
for key, group in groupby(test_file_name_list, key=lambda x: '-'.join(x.split('-')[:3])):
    #그룹 명(key) 목록
    print(key)
    groupby_key_list.append(key) 
    temp = list(group)
    
    #그룹화된 전체 테스트파일 목록 
    groupby_file_list.append(temp) 
    
    #그룹화 된 테스트 파일별 검출정보 목록(그룹화된 전체 테스트파일 목록과 동일한 구조로 구성),0 으로 초기화
    groupby_detected_number_for_files.append([0]*len(temp))

    #그룹화 된 테스트 파일별 검출된 최대 score정보 목록(그룹화된 전체 테스트파일 목록과 동일한 구조로 구성),0 으로 초기화
    #M_max_scores.append([0]*len(temp))
#print(test_file_name_list)
#print(len(groupby_file_list))
#print(groupby_file_list)

#annotation이 있는 파일 목록을 group by하여 list에 저장
#groupby key(그룹핑 기준(단위)는 파일명을 '-'로 split 한뒤 앞쪽 3번째 요소까지 추출함)
#파일 목록을 groupby 하기 위해서 정렬
'''
anno_file_list.sort(key=lambda x: '-'.join(x.split('-')[:3]))
for key, group in groupby(anno_file_list, key=lambda x: '-'.join(x.split('-')[:3])):
    #그룹 명(key) 목록
    temp = list(group)
    groupby_annotaion_file_dict[key] = temp
'''    
#print(groupby_annotaion_file_dict)    

## Run detection

In [ ]:
#set_anno_file_index()
model = []
model_count = 0;
for model_path in model_list:
    model_count += 1

    print('{} model {} load..'.format(ordinal(model_count), model_path.split('/')[-1]))
            
    # Load RetinaNet Models
    model.append(keras.models.load_model(model_path, custom_objects=custom_objects))

for SCORE_VALUE in SCORE_VALUE_LIST:
    print('****************{}*******************'.format(MODEL_ENV))
    print('*      Detecting with score {}           *'.format(SCORE_VALUE))
    print('*******************************************')
    
    print('{} test files.'.format(num_of_test_files))
        
    print("start.")
    start = time.time()
    for index_count, _fpath in enumerate(test_file_list):
        number_of_proper_detection = 0 #검출된 수
        b_anno_match = False
        b_is_annotation = False
        i_anno_detect = 0
        detect_list=[]
        detect_score_list=[]
        
        # load image
        #image, image_path = val_generator.load_image(index, get_image_path=True)
        if '_MEM_' in DATA_SOURCE_TYPE:
            image = jpg_generator.load_image(_fpath)
        elif '_FILE_' in DATA_SOURCE_TYPE:
            image = jpg_generator.load_file_image(_fpath)
        else:
            print('Define! Data source type...')
            break
        
        # copy to draw on
        draw = image.copy()
        draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
        
        # preprocess image for network
        #image = val_generator.preprocess_image(image)
        #image, scale = val_generator.resize_image(image)
        image = jpg_generator.preprocess_image(image)
        image, scale = jpg_generator.resize_image(image)
        #annotations = jpg_generator.read_annotations(_fpath)
        
        # for visualize annotations if imagae include annotation
        '''
        if len(annotations) > 0: 
            b_is_annotation = True #현재 파일이 annotation 정보를 가지고 있음
            anno_center_list = []
            for annotation in annotations:
                _center = []
                label = int(annotation[4])
                b = annotation[:4].astype(int)

                _center.append((b[0]+b[2])/2)
                _center.append((b[1]+b[3])/2)
                anno_center_list.append(_center)
                
                cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (50, 50, 255), 2)
                caption = "{}".format(jpg_generator.label_to_name(label))
                cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 0), 5)
                cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 4)
        '''
        for cur_model in model:
            # process detection
            _, _, detections = cur_model.predict_on_batch(np.expand_dims(image, axis=0))

            # compute predicted labels and scores
            predicted_labels = np.argmax(detections[0, :, 4:], axis=1)
            scores = detections[0, np.arange(detections.shape[1]), 4 + predicted_labels]

            # correct for image scale
            detections[0, :, :4] /= scale

            for idx, (label, score) in enumerate(zip(predicted_labels, scores)):
                if score < SCORE_VALUE or label != 0:
                    continue

                number_of_proper_detection += 1
                detect_list.append(detections[0, idx, :4].astype(int))
                detect_score_list.append(score)

        #이미지에서 (조건에 부합되게)detect된 것이 있을 경우
        if number_of_proper_detection > 0:
            #ensemble 모델들을 통해 얻은 선택 영역들에 대해 중복영역 등을 고려하여 최종 선택
            
            #1.검출영역 선택시 병합없이 선택
            #selected_detection_list_idxs = my.select_deteced_list(detect_list, detect_score_list)
            
            #2.검출영역 선택시 일정 조건에 따라 영역을 병합하며, Score는 병합된 영역의 평균으로 대체된다.
            selected_detection_list_idxs = my.select_deteced_list_wih_merge(detect_list, detect_score_list)
        
            for idxx, b in enumerate(detect_list):
                if selected_detection_list_idxs[idxx] == 0: #검출영역 정리되어 선택된 것만....
                    '''
                    if b_is_annotation and len(anno_center_list) > 0:
                        for anno_center in anno_center_list:
                            if b[0]<anno_center[0]<b[2] and b[1]<anno_center[1]<b[3]:
                                b_anno_match = True # annotation과 매칭되는 검출정보가 있음
                                i_anno_detect += 1
                                break
                    '''
                    #cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (0, 255, 0), 5)
                    cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (0, 255, 0), 2)
                    #caption = "{} {:.3f}".format(val_generator.label_to_name(label), detect_score_list[idxx])
                    caption = "{} {:.3f}".format(jpg_generator.label_to_name(label), detect_score_list[idxx])
                    cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 0), 5)
                    cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 4)

            #normal file에서 detect된 경우
            if 'normal' in _fpath:
                number_abnormal_detect_from_normal += 1            

            ##################### 그룹화된 테스트 파일 정보와 동일한 구조의 detection 정보 목록에 반영 ##############
            # 테스트 파일 그룹 목록에서 현재 파일의 위치를 구함    
            img_indics = my.find(groupby_file_list, make_file_name(_fpath))
            if len(img_indics) > 0:
                #groupby_detected_number_for_files[img_indics[0]][img_indics[1]] += number_of_proper_detection
                groupby_detected_number_for_files[img_indics[0]][img_indics[1]] = 1
                
             # annotation 검출결과(개수 등) 저장
            '''
            if b_is_annotation:
                annotation_detect_num_dict[make_file_name(_fpath)] = i_anno_detect
            '''
        ############검출된 정보를 이미지에 표시, 저장################
        #1. 모든 파일저장, annotation 유무에 관계없이, 검출영역 정보 표시
        #cv2.imwrite(os.path.join(result_save_path, '-'.join(image_path.split('/')[-2:])), draw)
        cv2.imwrite(os.path.join(result_save_path, make_file_name(_fpath)+'.jpg'), draw)
        
        #2. annotation 정보를 가지고 있는 파일만 저장, 검출 결과도 함께 표시
        '''
        if b_is_annotation:
            #cv2.imwrite(os.path.join(result_save_path, '-'.join(image_path.split('/')[-2:])), draw)
            cv2.imwrite(os.path.join(result_save_path, make_file_name(_fpath)+'.jpg'), draw)

            #annotation 미 검출(annotation 검출 조건이 1개도 만족되지 않음)된 파일만 따로 저장
            if not b_anno_match:
                # save the image into file
                cv2.imwrite(os.path.join(result_anno_miss_save_path, make_file_name(_fpath)+'.jpg'), draw)
        '''
        del detect_list
        del detect_score_list

        #print('{}/{} completed.\r'.format(index+1,num_of_test_files), end='')
        print('{}/{} completed.\r'.format(index_count+1,num_of_test_files), end='')

    print("end({:.2f} minutes elapsed).".format((time.time() - start)/60))

    # release the memory
    #del model
    #keras.backend.clear_session()
        
    get_detection_info() #detection 결과 출력
   
    print('\n')
    
    #다음 Score로 테스트하기 위해 이전 검출 정보를 초기화 해줘야 함...
    init_info()
    

In [ ]:
print('Finished!')